In [1]:
import os
import tensorflow as tf
import numpy as np

from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt

from tensorflow import keras
import tensorflow_datasets as tfds
from keras.datasets import fashion_mnist

tfds.disable_progress_bar()

from tensorflow.keras.models import Model
from tensorflow.keras.losses import Loss
from tensorflow.keras.layers import Layer,Lambda,InputLayer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

import cvnn.layers as complex_layers

import warnings
warnings.filterwarnings('ignore')

In [2]:
# pip uninstall tensorflow
# pip install tensorflow==2.9.0 cvnn tensorflow_datasets

In [3]:
v = tf.Variable(1.)
casted = tf.cast(v,dtype=tf.complex64)
print(casted)

print(tf.math.exp(1j*tf.cast(v,dtype=tf.complex64)))
print(tf.math.exp(tf.cast(v,dtype=tf.complex64)))
print(tf.complex(tf.cos(v),tf.sin(v)))


tf.Tensor((1+0j), shape=(), dtype=complex64)
tf.Tensor((0.5403023+0.841471j), shape=(), dtype=complex64)
tf.Tensor((2.7182817+0j), shape=(), dtype=complex64)
tf.Tensor((0.5403023+0.841471j), shape=(), dtype=complex64)


2022-07-27 00:50:57.692249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 00:50:57.700781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 00:50:57.701013: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 00:50:57.702018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
size = 200
BUFFER_SIZE = 5000
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

# Define the traning parameters
keep_training = False
learning_rate = 0.01
epochs = 20

# load the dataset
datasets, info = tfds.load(name='fashion_mnist', with_info=True, as_supervised=True, data_dir='./data')

fashion_mnist_train, fashion_mnist_test = datasets['train'], datasets['test']

num_train = info.splits['train'].num_examples
num_test = info.splits['test'].num_examples

### 1.Define phase object ( Preprocess the images with up-sampling )
1. Load image that serves as phase object  

2. Image is 28x28 pixels, and is padded to 200x200 pixels  with 0's  

3. Phase Image = exp(2$\pi$ i * Padded Image)

The digital image is encoded in phasor form, with an uniform amplitude and different phase angle.


In [5]:
def preprocess(image, label):
    label = tf.one_hot(tf.cast(label, tf.int32), 10)   # convert the label to categorial, or one-hot coded
    
    up_sampling_size = int(1*size)
    padding_size = (size - up_sampling_size)//2
    image = tf.cast(image, tf.float32)
    # Step1: upsample the image to 120x120
    up_sampling_image = tf.image.resize(image,
                                        size=[up_sampling_size,up_sampling_size],
                                        method='nearest')
    up_sampling_image = up_sampling_image / 255.0
    # Step2: get the phase object
    phase_image = tf.math.exp(2*np.pi*1j*tf.cast(up_sampling_image,dtype=tf.complex64))
    # Step3: pad the phase object to 200x200 with 0s
    zero_padded_image = tf.pad(phase_image,
                                paddings=[[padding_size,padding_size],[padding_size,padding_size],[0,0]],
                                mode="CONSTANT",constant_values=0)
        
    return tf.cast(zero_padded_image, dtype=tf.complex64), label
    
train_dataset = fashion_mnist_train.map(preprocess).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = fashion_mnist_test.map(preprocess).batch(BATCH_SIZE)

##### Notice:
For ___tf.pad()___,  paddings中的第一个[40,40]表示的是垂直方向上的填充，第一个40表示的是向上的填充行数，第二个40表示的是向下的填充行数。第二个[40,40]表示的水平方向上的填充，第一个40表示的向左的填充行数，第二个40表示的是向右的填充行数。

### 2.Build the Diffraction layer using Angular Spectrum method

### Angular Spectrum Propagation
$$U_1(x,y) =\mathcal{F}^{-1}[\mathcal{F} U_0(x,y)\mathcal{F}h(x,y)]$$

$$U_1(x,y) =\mathcal{F}^{-1}[\mathcal{F} U_0(x,y) H(f_x,f_y)]$$


This can be described using Fourier transforms.The first Fourier transform decomposes the initial field into plane waves. To propagate the plane waves, we multiply each wave by a complex phase factor, and then we take the inverse Fourier transform to add all the propagated plane waves back together.

To implement Angular spectrum propagation, the Fouier transform of the initial field is first multiplied with the phase factor $$H=e^{ik_zz}$$, where $k_z$is a function of the spatial frequencies $$k_z=\sqrt{k^2-k_x^2-k_y}$$where $$  k = \frac{2\pi}{\lambda}$$ and $k_x$ and $k_y$ are related to the spatial frequencies $f_x$ and $f_y$ by a factor of $2\pi$ $$k_{x,y} = 2\pi f_{x,y}$$

Hence the complex exponential can be written in terms of the Fourier coordinates $f_x$ and $f_y$, that is 

$$H=e^{ik_zz},k_z = 2\pi \sqrt{\frac{1}{\lambda}-f_x-f_y}$$

Descretized spatial freqnency $f_x = k*\Delta f = \frac{k}{N \Delta x}$  
Interval between the spatial frequencies $\Delta f = \frac{1}{N \Delta x}=\frac{1}{L}$,where $L$ denotes the field of view in object space

In [6]:
class Diffraction_Layer(Layer):
    def __init__(self, units =200):
        '''Initialize the diffraction layer attributes'''
        super(Diffraction_Layer, self).__init__()
        self.units = units
        self.Nx = units              # Nx is the dimension of the grid
        # self.L = 0.08              # source and observation plane side length, field of view
        self.dx = 2e-6
        self.lam = 7.5e-6          # wavelength of the optical wave
        self.z = 3e-2                # distance of propagation(the distance bewteen two layers)
   
    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        phase_init = tf.random_normal_initializer()
        self.phase = tf.Variable(name= "phase",
                                initial_value=phase_init(shape=(self.units,self.units), dtype='float32'),
                                trainable=True)
                                # constraint=lambda t: 2*np.pi*tf.math.sigmoid(t))
        # To help with the 3D-printing and fabrication of the D2NN design, 
        # a sigmoid function was used to constrain the phase value of each neuron
    @tf.function
    def call(self, inputs):
        '''Define the computation'''
        def angular_spectrum_propagator(E, z = self.z, lam = self.lam):
            # compute angular spectrum
            fft_c = tf.signal.fft2d(E)
            c = tf.signal.fftshift(fft_c)

            fx = np.fft.fftshift(np.fft.fftfreq(self.Nx, d = self.dx))
            fxx, fyy = np.meshgrid(fx, fx)
            argument = (2 * np.pi)**2 * ((1. / lam) ** 2 - fxx ** 2 - fyy ** 2)

           #Calculate the propagating and the evanescent (complex) modes
            tmp = np.sqrt(np.abs(argument))
            kz = np.where(argument >= 0, tmp, 1j*tmp)

            # propagate the angular spectrum a distance z
            E = tf.signal.ifft2d(tf.signal.ifftshift(c * np.exp(1j * kz * z)))
            return E
        return tf.multiply(angular_spectrum_propagator(inputs),tf.math.exp(1j*tf.cast(self.phase,dtype=tf.complex64)))

#### Notice
For ___tf.cast()___: In case of casting from real types to complex types(complex64), the imaginary part of the returned value is set to 0.    

In [7]:
class Propogation(Layer):
    def __init__(self, units =200):
        '''Initialize the diffraction layer attributes'''
        super(Propogation, self).__init__()
        self.units = units
        self.Nx = units              # Nx is the dimension of the grid
        self.dx = 2e-6
        self.lam = 7.5e-6            # wavelength of the optical wave
        self.z = 1e-2                # distance of propagation(the distance bewteen last layer and the detector)
        
    @tf.function
    def call(self, inputs):
        '''Define the computation'''
        def angular_spectrum_propagator(E, z = self.z, lam = self.lam):
            # compute angular spectrum
            fft_c = tf.signal.fft2d(E)
            c = tf.signal.fftshift(fft_c)

            fx = np.fft.fftshift(np.fft.fftfreq(self.Nx, d = self.dx))
            fxx, fyy = np.meshgrid(fx, fx)
            argument = (2 * np.pi)**2 * ((1. / lam) ** 2 - fxx ** 2 - fyy ** 2)

           #Calculate the propagating and the evanescent (complex) modes
            tmp = np.sqrt(np.abs(argument))
            kz = np.where(argument >= 0, tmp, 1j*tmp)

            # propagate the angular spectrum a distance z
            E = tf.signal.ifft2d(tf.signal.ifftshift(c * np.exp(1j * kz * z))) # phase是加还是减
            return E
        return angular_spectrum_propagator(inputs)
        

In [8]:
class Detector(Layer):
    def __init__(self, units=200):
        '''Initialize the instance attributes'''
        super(Detector, self).__init__()
        self.units = units
    
    @tf.function
    def call(self, inputs):
        ''' Converts output to one hot form
        Applies softmax'''
        
        def rang(arr,shape,size=size,base = 512):
            x0 = shape[0] * size // base
            y0 = shape[2] * size // base
            delta = (shape[1]-shape[0])* size // base
            return arr[x0:x0+delta,y0:y0+delta]
        
        def reduce_mean(tf_):
            return tf.reduce_mean(tf_)
        
        def _ten_regions(a):
            return tf.map_fn(reduce_mean,tf.convert_to_tensor([
                rang(a,(120,170,120,170)),
                rang(a,(120,170,240,290)),
                rang(a,(120,170,360,410)),
                rang(a,(220,270,120,170)),
                rang(a,(220,270,200,250)),
                rang(a,(220,270,280,330)),
                rang(a,(220,270,360,410)),
                rang(a,(320,370,120,170)),
                rang(a,(320,370,240,290)),
                rang(a,(320,370,360,410))
            ]))
        
        def ten_regions(logits):
            return tf.map_fn(_ten_regions,tf.abs(logits),dtype=tf.float32)

        return tf.square(ten_regions(tf.abs(inputs))) # logits_abs

In [9]:
def loss_function(y_label,logits_abs):
    return tf.reduce_mean(tf.square(logits_abs-y_label))

In [10]:
def get_D2NN_model():
    inputs = complex_layers.complex_input(shape=(size,size))
    h1 = Diffraction_Layer(size)(inputs)
    h2 = Diffraction_Layer(size)(h1)
    h3 = Diffraction_Layer(size)(h2)
    h4 = Diffraction_Layer(size)(h3)
    h5 = Diffraction_Layer(size)(h4)
    propogation = Propogation(size)(h5)
    out = Detector()(propogation)
    return tf.keras.Model(inputs, out)


D2NN = get_D2NN_model()

D2NN.summary()
# plot_model(D2NN, show_shapes=True, show_layer_names=True, to_file='D2NN-model.png')

Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (ComplexInput)      [(None, 200, 200)]        0         
                                                                 
 diffraction__layer (Diffrac  (None, 200, 200)         40000     
 tion_Layer)                                                     
                                                                 
 diffraction__layer_1 (Diffr  (None, 200, 200)         40000     
 action_Layer)                                                   
                                                                 
 diffraction__layer_2 (Diffr  (None, 200, 200)         40000     
 action_Layer)                                                   
                                                                 
 diffraction__layer_3 (Diffr  (None, 200, 200)         40000     
 action_Layer)                                               

In [ ]:
if keep_training:
    D2NN.load_weights('./Training_results/D2NN_MODEL.63-7.3470')

D2NN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
             loss=loss_function,
             metrics=['accuracy'])

checkpoint_path = './Training_results/D2NN_MODEL.{epoch}-{val_loss:.4f}'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             sace_freq='epoch')

history = D2NN.fit(train_dataset,
                   epochs=epochs,
                   validation_data=test_dataset,
                   callbacks=[checkpoint])

Epoch 1/20
938/938 [==============================] - 320s 336ms/step - loss: 0.0811 - accuracy: 0.6440 - val_loss: 0.0479 - val_accuracy: 0.7764
Epoch 2/20
938/938 [==============================] - 292s 311ms/step - loss: 0.0431 - accuracy: 0.7981 - val_loss: 0.0412 - val_accuracy: 0.7941
Epoch 3/20
938/938 [==============================] - ETA: 0s - loss: 0.0391 - accuracy: 0.8134

|  num   | dataset|type  |dx  |lam |z|accuracy|
|  ----  | ----  | ----  | ----  | ----  | ----  | ----  |
|  1 | fminst|amplitude  | 2e-6  |7.5e-6  |0.01|73%|

#### Extracting weights from model layers

In [ ]:
# print(D2NN.layers)
# print(D2NN.layers[1].weights)
# print(D2NN.layers[2].get_weights()) # get the numpy arrays for the parameters of the layer
# print(D2NN.get_layer('diffraction__layer_1').phase)